In [1]:
import fsspec
import time
from pyspark.sql import SparkSession
import json
from functools import reduce
from pyspark.sql.functions import col,when,lit,round,coalesce,unix_timestamp,year,month,trim,upper,create_map,abs,greatest,to_date,current_date
from datetime import datetime,timedelta
from shapely.geometry import Point
from shapely.wkt import loads as wkt_loads
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, TimestampType
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
spark=SparkSession.builder.appName('nyc_to_silver').getOrCreate()
abfss_path_bronze = "abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/nyc"
checkpoint_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/8d633764-c954-4f64-a6ff-b52dde76bd20/Files/checkpoint/checkpoint.json"
zone_lookup="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/8d633764-c954-4f64-a6ff-b52dde76bd20/Tables/silver/taxi_zones"
account_name = "4906b11e-1e59-4869-9321-062a4696a2db"
account_host = "onelake.dfs.fabric.microsoft.com"


fs = fsspec.filesystem(
    "abfss",
    account_name=account_name,
    account_host=account_host
)
with fs.open(f"{checkpoint_path}") as f:
    checkpoint=json.load(f)



StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
def latest_date() -> dict:
    """
    returns dict object max(year,month) for taxi_type
    """
    base_path="abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/nyc/data"
    taxi_types=[y.name for y in mssparkutils.fs.ls(base_path)]
    taxi_dict={t:None for t in taxi_types}
    for t in taxi_types:
        years = [int(y.name) for y in mssparkutils.fs.ls(f"{base_path}/{t}")]
        year = max(years)
        months = [int(m.name) for m in mssparkutils.fs.ls(f"{base_path}/{t}/{year}")]
        month = max(months)
        taxi_dict[t]={'year':year,'month':month}

    return taxi_dict

def add_one_month(dt):
    """
    adds 1 months to current dt
    """
    year = dt.year + (dt.month // 12)
    month = (dt.month % 12) + 1
    return dt.replace(year=year, month=month)


def generate_paths(taxi_type,start,end):
    """
    generates paths for taxi_type from start to end 
    """
    if isinstance(start, str):
        start = datetime.fromisoformat(start)
    if isinstance(end, str):
        end = datetime.fromisoformat(end)

    paths = []
    curr = start
    if start == end:
        return paths

    while curr <= end:
        paths.append(
            f"{abfss_path_bronze}/data/"
            f"{taxi_type}/"
            f"{curr.year}/"
            f"{curr.month:02d}/*.parquet"
        )
        curr = add_one_month(curr)
    return paths




StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
def transform_df(df):
    df = df.withColumn(
        "trip_duration",
        (col("dropoff_datetime").cast("long") - col("pickup_datetime").cast("long")) / 60
    )
    df = df.withColumn(
        "avg_speed_mph",
        when(
            (col("trip_duration") > 0) & col("trip_distance").isNotNull(),
            col("trip_distance") / (col("trip_duration") / 60)
        ).otherwise(lit(None))
    )
    df=df.withColumn(
        "RatecodeID",
        when(col("RatecodeID").isNull(),lit(99))
        .otherwise(col('RatecodeID').cast('int'))
    )
    return df

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
ignore_cols = {
    "ehail_fee",
    "trip_type",
    "airport_fee",
    "cbd_congestion_fee",
    "Airport_fee"
}

def read_and_save(paths, taxi_type):
    table_name = f"silver.taxi_{taxi_type}"
    mode = "overwrite" if not spark.catalog.tableExists(table_name) else "append"

    for p in paths:
        df = spark.read.parquet(p)
        if taxi_type == "yellow":
            df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
                   .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
        else:
            df = df.withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
                   .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

        df = df.drop(*[c for c in ignore_cols if c in df.columns])
        casts = {
            "VendorID": "string",
            "RatecodeID": "string",
            "payment_type": "string",
            "store_and_fwd_flag": "string",
            "PULocationID": "long",
            "DOLocationID": "long",
            "pickup_datetime": "timestamp",
            "dropoff_datetime": "timestamp",
            "passenger_count": "long",
            "trip_distance": "double",
            "fare_amount": "double",
            "extra": "double",
            "mta_tax": "double",
            "tip_amount": "double",
            "tolls_amount": "double",
            "improvement_surcharge": "double",
            "congestion_surcharge": "double",
            "total_amount": "double"
        }

        for c, t in casts.items():
            if c in df.columns:
                df = df.withColumn(c, col(c).cast(t))

        df = df.filter(
            (col("pickup_datetime").isNotNull()) &
            (col("dropoff_datetime").isNotNull()) &
            (col("dropoff_datetime") > col("pickup_datetime")) &  
            (col("trip_distance") > 0) &                       
            (col("dropoff_datetime") <= current_date())        
        )

        df = df.withColumn("fare_amount_abs", abs(col("fare_amount"))) \
               .withColumn("total_amount_net", greatest(col("total_amount"), lit(0.0))) \
               .withColumn("pickup_date", to_date(col("pickup_datetime"))) \
               .withColumn("dropoff_date", to_date(col("dropoff_datetime")))


        df = transform_df(df)
        df = df.filter(
            (col("pickup_date") >= lit("2014-01-01")) &
            (col("pickup_date") <= current_date())
        )

        df = df.withColumn("year", year(col("pickup_date"))) \
               .withColumn("month", month(col("pickup_date")))
        df.write.format("delta") \
          .mode(mode) \
          .partitionBy("year", "month") \
          .saveAsTable(table_name)

        mode = 'append'

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
to_silver = checkpoint.get("taxi", {})
green_start = to_silver.get("green", datetime(2014,1,1))
yellow_start = to_silver.get("yellow", datetime(2014,1,1))
latest = latest_date()

yellow_end = datetime(latest['yellow']['year'], latest['yellow']['month'], 1)
green_end = datetime(latest['green']['year'], latest['green']['month'], 1)

yellow_path = generate_paths("yellow", yellow_start, yellow_end)
green_path = generate_paths("green", green_start, green_end)


StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
read_and_save(yellow_path,'yellow')
read_and_save(green_path,'green')

StatementMeta(, , -1, Cancelled, , Cancelled)